In [1]:
%%bash
set -euxo pipefail
sudo apt-get update -y
sudo apt-get install -y \
  python3.10 python3.10-venv python3.10-dev \
  git ninja-build build-essential \
  libgl1 libglib2.0-0 ffmpeg


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,361 kB]
Get:10 https://cli.github.com/packages stable/main amd64 Packages [355 B]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,904 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restrict

+ sudo apt-get update -y
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
+ sudo apt-get install -y python3.10 python3.10-venv python3.10-dev git ninja-build build-essential libgl1 libglib2.0-0 ffmpeg
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 5.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [2]:
%%bash
set -euxo pipefail
python3.10 -m venv /content/py310
source /content/py310/bin/activate
python -V
pip install -U pip setuptools wheel


Python 3.10.12
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 KB 14.2 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 68.1.2
    Uninstalling setuptools-68.1.2:
      Successfully uninstalled setuptools-68.1.2
  Attempting uninstall: pip
    Found existing installation: pip 22.0.2
    Uninstalling pip-22.0.2:
      Successfully uninstalled pip-22.0.2


+ python3.10 -m venv /content/py310
+ source /content/py310/bin/activate
++ deactivate nondestructive
++ '[' -n '' ']'
++ '[' -n '' ']'
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
++ '[' -n '' ']'
++ unset VIRTUAL_ENV
++ unset VIRTUAL_ENV_PROMPT
++ '[' '!' nondestructive = nondestructive ']'
++ VIRTUAL_ENV=/content/py310
++ export VIRTUAL_ENV
++ _OLD_VIRTUAL_PATH=/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ PATH=/content/py310/bin:/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ export PATH
++ '[' -n '' ']'
++ '[' -z '' ']'
++ _OLD_VIRTUAL_PS1=
++ PS1='(py310) '
++ export PS1
++ VIRTUAL_ENV_PROMPT='(py310) '
++ export VIRTUAL_ENV_PROMPT
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
+ python -V
+ pip install -U pip setuptools wheel


In [3]:
%%bash
set -euxo pipefail
cd /content
git clone --recurse-submodules https://github.com/DopeMeerkat/TRELLIS.git
cd /content/TRELLIS
git submodule update --init --recursive


Submodule path 'trellis/representations/mesh/flexicubes': checked out '815e075a2a400d06c48d94c347674344ed6ae5c5'


+ cd /content
+ git clone --recurse-submodules https://github.com/DopeMeerkat/TRELLIS.git
Cloning into 'TRELLIS'...
Submodule 'trellis/representations/mesh/flexicubes' (https://github.com/MaxtirError/FlexiCubes.git) registered for path 'trellis/representations/mesh/flexicubes'
Cloning into '/content/TRELLIS/trellis/representations/mesh/flexicubes'...
+ cd /content/TRELLIS
+ git submodule update --init --recursive


In [9]:
%%bash
set -euxo pipefail
source /content/py310/bin/activate

# Torch
pip uninstall -y torch torchvision torchaudio || true
pip install --index-url https://download.pytorch.org/whl/cu121 \
  torch==2.5.1+cu121 torchvision==0.20.1+cu121 torchaudio==2.5.1+cu121

python -c "import torch; print('torch', torch.__version__); print('cuda', torch.version.cuda); print('avail', torch.cuda.is_available()); print('gpu', torch.cuda.get_device_name(0) if torch.cuda.is_available() else None)"


# Build tools + common deps
pip install -U packaging ninja cmake psutil

pip install "kaolin==0.18.0" -f "https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-2.5.1_cu121.html"

python -c "import kaolin; print('kaolin', kaolin.__version__)"

# flash-attn (T4 is SM75; build for it)
export CUDA_HOME=/usr/local/cuda
export PATH="$CUDA_HOME/bin:$PATH"
export TORCH_CUDA_ARCH_LIST="7.5"
pip uninstall -y flash-attn || true
pip install --no-build-isolation -v flash-attn==2.8.3

python -c "import flash_attn; print('flash_attn', flash_attn.__version__)"


Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.5.1%2Bcu121-cp310-cp310-linux_x86_64.whl (780.4 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchvision-0.20.1%2Bcu121-cp310-cp310-linux_x86_64.whl (7.3 MB)
  Using cached https://download.pytorch.org/whl/cu121/torchaudio-2.5.1%2Bcu121-cp310-cp310-linux_x86_64.whl (3.4 MB)

torch 2.5.1+cu121
cuda 12.1
avail True
gpu NVIDIA A100-SXM4-40GB
Looking in links: https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-2.5.1_cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━

+ source /content/py310/bin/activate
++ deactivate nondestructive
++ '[' -n '' ']'
++ '[' -n '' ']'
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
++ '[' -n '' ']'
++ unset VIRTUAL_ENV
++ unset VIRTUAL_ENV_PROMPT
++ '[' '!' nondestructive = nondestructive ']'
++ VIRTUAL_ENV=/content/py310
++ export VIRTUAL_ENV
++ _OLD_VIRTUAL_PATH=/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ PATH=/content/py310/bin:/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ export PATH
++ '[' -n '' ']'
++ '[' -z '' ']'
++ _OLD_VIRTUAL_PS1=
++ PS1='(py310) '
++ export PS1
++ VIRTUAL_ENV_PROMPT='(py310) '
++ export VIRTUAL_ENV_PROMPT
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
+ pip uninstall -y torch torchvision torchaudio
+ pip install --index-url https://download.pytorch.org/whl/cu121 torch==2.5.1+cu121 torchvision==0.20.1+cu121 t

In [5]:
%%bash
set -euxo pipefail
source /content/py310/bin/activate
cd /content/TRELLIS

# prevent extension collisions between runs
rm -rf /tmp/extensions || true

# TRELLIS setup for basic deps + spconv backend
source ./setup.sh --basic --spconv

# nvdiffrast (match your local v0.3.4)
pip install --no-build-isolation "git+https://github.com/NVlabs/nvdiffrast.git@v0.3.4"

python - <<'PY'
import importlib.metadata as md, torch
print("[torch]", torch.__version__)
try:
    import spconv
    print("[spconv] OK")
except Exception as e:
    print("[spconv] FAIL", e)

import nvdiffrast, os
print("[nvdiffrast]", md.version("nvdiffrast"), os.path.dirname(nvdiffrast.__file__))
PY


[SYSTEM] PyTorch Version: 2.5.1+cu121, CUDA Version: 12.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 42.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 MB 138.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 181.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 196.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 740.3/740.3 kB 40.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 48.4 MB/s  0:00:06
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 130.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.6/145.6 MB 145.5 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 74.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 137.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 175.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 33.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━

+ source /content/py310/bin/activate
++ deactivate nondestructive
++ '[' -n '' ']'
++ '[' -n '' ']'
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
++ '[' -n '' ']'
++ unset VIRTUAL_ENV
++ unset VIRTUAL_ENV_PROMPT
++ '[' '!' nondestructive = nondestructive ']'
++ VIRTUAL_ENV=/content/py310
++ export VIRTUAL_ENV
++ _OLD_VIRTUAL_PATH=/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ PATH=/content/py310/bin:/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ export PATH
++ '[' -n '' ']'
++ '[' -z '' ']'
++ _OLD_VIRTUAL_PS1=
++ PS1='(py310) '
++ export PS1
++ VIRTUAL_ENV_PROMPT='(py310) '
++ export VIRTUAL_ENV_PROMPT
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
+ cd /content/TRELLIS
+ rm -rf /tmp/extensions
+ source ./setup.sh --basic --spconv
+++ getopt -o h --long help,new-env,basic,train,xformers,flash-attn,diffoct

In [6]:
%%bash
set -euxo pipefail
source /content/py310/bin/activate

python - <<'PY'
from pathlib import Path
import re

p = Path("/content/TRELLIS/trellis/pipelines/base.py")
s = p.read_text()

# Ensure os import exists
if not re.search(r"^\s*import os\s*$", s, re.M):
    s = re.sub(r"(?m)^((?:from .+\n|import .+\n)+)", r"\1import os\n", s, count=1)

needle = 'config_file = hf_hub_download(path, "pipeline.json")'
m = re.search(r"(?m)^(?P<indent>\s*)" + re.escape(needle) + r"\s*$", s)
if not m:
    raise SystemExit("Expected line not found: " + needle)

indent = m.group("indent")
replacement = "\n".join([
    f"{indent}# Local folder support: if `path` is a directory, read pipeline.json from disk",
    f"{indent}if os.path.isdir(path):",
    f"{indent}    config_file = os.path.join(path, 'pipeline.json')",
    f"{indent}    if not os.path.exists(config_file):",
    f"{indent}        raise FileNotFoundError('Missing pipeline.json in local folder: ' + str(path))",
    f"{indent}else:",
    f"{indent}    config_file = hf_hub_download(path, 'pipeline.json')",
])

s = re.sub(r"(?m)^" + re.escape(indent) + re.escape(needle) + r"\s*$", replacement, s, count=1)
p.write_text(s)
print("Patched:", p)
PY

/content/py310/bin/python -m py_compile /content/TRELLIS/trellis/pipelines/base.py
echo "base.py compiles"


Patched: /content/TRELLIS/trellis/pipelines/base.py
base.py compiles


+ source /content/py310/bin/activate
++ deactivate nondestructive
++ '[' -n '' ']'
++ '[' -n '' ']'
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
++ '[' -n '' ']'
++ unset VIRTUAL_ENV
++ unset VIRTUAL_ENV_PROMPT
++ '[' '!' nondestructive = nondestructive ']'
++ VIRTUAL_ENV=/content/py310
++ export VIRTUAL_ENV
++ _OLD_VIRTUAL_PATH=/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ PATH=/content/py310/bin:/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ export PATH
++ '[' -n '' ']'
++ '[' -z '' ']'
++ _OLD_VIRTUAL_PS1=
++ PS1='(py310) '
++ export PS1
++ VIRTUAL_ENV_PROMPT='(py310) '
++ export VIRTUAL_ENV_PROMPT
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
+ python -
+ /content/py310/bin/python -m py_compile /content/TRELLIS/trellis/pipelines/base.py
+ echo 'base.py compiles'


In [7]:
%%bash
set -euxo pipefail
source /content/py310/bin/activate

pip install -U huggingface_hub

python - <<'PY'
from huggingface_hub import snapshot_download
path = snapshot_download(
    repo_id="microsoft/TRELLIS-image-large",
    local_dir="/content/TRELLIS-image-large",
    local_dir_use_symlinks=False,
)
print("downloaded to:", path)
PY

# Use local absolute folder
sed -i 's|from_pretrained("./TRELLIS-image-large")|from_pretrained("/content/TRELLIS-image-large")|g' /content/TRELLIS/generate.py


downloaded to: /content/TRELLIS-image-large


+ source /content/py310/bin/activate
++ deactivate nondestructive
++ '[' -n '' ']'
++ '[' -n '' ']'
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
++ '[' -n '' ']'
++ unset VIRTUAL_ENV
++ unset VIRTUAL_ENV_PROMPT
++ '[' '!' nondestructive = nondestructive ']'
++ VIRTUAL_ENV=/content/py310
++ export VIRTUAL_ENV
++ _OLD_VIRTUAL_PATH=/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ PATH=/content/py310/bin:/opt/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
++ export PATH
++ '[' -n '' ']'
++ '[' -z '' ']'
++ _OLD_VIRTUAL_PS1=
++ PS1='(py310) '
++ export PS1
++ VIRTUAL_ENV_PROMPT='(py310) '
++ export VIRTUAL_ENV_PROMPT
++ '[' -n /usr/bin/bash -o -n '' ']'
++ hash -r
+ pip install -U huggingface_hub
+ python -
/content/py310/lib/python3.10/site-packages/huggingface_hub/utils/_validators.py:202: UserWarning: The `local_dir_u

In [10]:
%%bash
set -euxo pipefail
mkdir -p /content/outputs
/content/py310/bin/python /content/TRELLIS/generate.py /content/hambaga.jpg /content/outputs/hambaga


[SPARSE] Backend: spconv, Attention: flash_attn
[SPARSE][CONV] spconv algo: native
[ATTENTION] Using backend: flash_attn
3D generation completed successfully!
Generated 1 Gaussian splat(s)
Generated 1 radiance field(s)
Generated 1 mesh(es)
Saved raw mesh OBJ: /content/outputs/hambaga_raw.obj
Done. Note: Rendering videos is omitted to keep the script lightweight for Colab.


+ mkdir -p /content/outputs
+ /content/py310/bin/python /content/TRELLIS/generate.py /content/hambaga.jpg /content/outputs/hambaga
/content/py310/lib/python3.10/site-packages/spconv/pytorch/functional.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  _TORCH_CUSTOM_FWD = amp.custom_fwd(cast_inputs=torch.float16)
/content/py310/lib/python3.10/site-packages/spconv/pytorch/functional.py:97: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/content/py310/lib/python3.10/site-packages/spconv/pytorch/functional.py:163: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):
/content/py310/lib/python3.10/site-packages/spconv/pytorch/functional.py:243: FutureWarn